## Gauss/seidel para Sistemas não Lineares

O método de Gauss-Seidel para sistemas não lineares resolve iterativamente o sistema, utilizando os valores mais recentes disponíveis para acelerar a convergência.

In [1]:
import numpy as np
import sympy as sp
from tabulate import tabulate

In [7]:
def gauss_seidel_sistemas(funcs, vars, x0, tol, max_iter=100):
    """
    Resolve sistemas de equações não lineares pelo método de Gauss-Seidel.
    
    Parâmetros:
    funcs -- Lista de funções do sistema (expressões simbólicas)
    vars -- Lista de variáveis simbólicas
    x0 -- Aproximação inicial (array ou lista)
    tol -- Erro de tolerância
    max_iter -- Número máximo de iterações (padrão: 100)
    
    Retorna:
    Solução aproximada e tabela com as iterações
    """
    # Isola cada variável em suas respectivas funções
    isolated_funcs = []
    for i, func in enumerate(funcs):
        # Isola a variável vars[i] na função func
        isolated_var = sp.solve(func, vars[i])
        if not isolated_var:
            raise ValueError(f"Não foi possível isolar a variável {vars[i]} na função {func}.")
        isolated_funcs.append(sp.lambdify(vars, isolated_var[0], 'numpy'))
    
    # Inicializa a solução inicial
    x = np.array(x0, dtype=float)
    tabela = []
    
    for iteration in range(max_iter):
        x_old = x.copy()
        
        # Atualização sequencial de Gauss-Seidel
        for i in range(len(vars)):
            x[i] = isolated_funcs[i](*x)  # Atualiza x[i] usando a função correspondente
        
        # Calcula os erros absoluto e relativo
        erro_absoluto = np.linalg.norm(x - x_old)
        erro_relativo = erro_absoluto / (np.linalg.norm(x) if np.linalg.norm(x) != 0 else 1)
        
        # Armazena os resultados na tabela
        tabela.append([iteration, *np.round(x, 4), round(erro_absoluto, 4), round(erro_relativo, 4)])
        
        # Verifica a condição de parada
        if erro_absoluto <= tol:
            return np.round(x, 4), tabela  # Arredonda a solução final para 4 casas decimais
    
    print("O método não convergiu dentro do número máximo de iterações.")
    return np.round(x, 4), tabela

In [9]:
# Entrada do usuário
n = int(input("Digite o número de equações/variáveis: "))
vars = sp.symbols(f'x1:{n+1}')
# funcs = []
funcs = [sp.sympify("5*x1+x2+x3-50"), sp.sympify("-x1+3*x2-x3-10"), sp.sympify("x1+2*x2+10*x3+30")]

# print("Digite as funções do sistema:")
# for i in range(n):
    # funcs.append(sp.sympify(input(f"f{i+1}(x1, ..., xn) = ")))

x0 = [float(input(f"Digite x{i+1} inicial: ")) for i in range(n)]
tol = float(input("Digite o erro de tolerância: "))

solucao, tabela = gauss_seidel_sistemas(funcs, vars, x0, tol)

# Exibindo tabela de resultados
headers = ["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro Absoluto", "Erro Relativo"]
print(tabulate(tabela, headers=headers, floatfmt=".4f"))
print(f"Solução aproximada: {solucao}")

  Iteração       x1      x2       x3    Erro Absoluto    Erro Relativo
----------  -------  ------  -------  ---------------  ---------------
         0  10.0000  6.6667  -5.3333          13.1487           1.0000
         1   9.7333  4.8000  -4.9333           1.9276           0.1617
         2  10.0267  5.0311  -5.0089           0.3810           0.0310
         3   9.9956  4.9956  -4.9987           0.0483           0.0039
         4  10.0006  5.0007  -5.0002           0.0073           0.0006
         5   9.9999  4.9999  -5.0000           0.0011           0.0001
         6  10.0000  5.0000  -5.0000           0.0002           0.0000
         7  10.0000  5.0000  -5.0000           0.0000           0.0000
Solução aproximada: [10.  5. -5.]
